In [7]:
import pandas as pd

In [8]:
nowon= pd.read_pickle("C:/Users/user/Finance/bigcon/기상/nowon_365.pickle")
jongro = pd.read_pickle("C:/Users/user/Finance/bigcon/기상/jongro_365.pickle")

In [9]:
# 종로구, 노원구 전체
gu_total = pd.concat([nowon, jongro])
del gu_total['serial']
gu_total

,행정동,date,pm10,pm10_class,pm25,pm25_class,pm_class,pm_class_info
0,상계5동,2018-04-01,92.3,4,63.2,2,2,pm10
1,상계2동,2018-04-01,87.4,4,59.2,2,2,pm10
2,상계1동,2018-04-01,72.0,2,34.7,1,1,pm10
3,상계2동,2018-04-01,81.0,4,58.9,2,2,pm10
4,상계2동,2018-04-01,77.5,2,42.8,2,2,pm25
5,공릉2동,2018-04-01,77.8,2,41.2,2,2,pm25
6,상계10동,2018-04-01,85.5,4,59.6,2,2,pm10
7,상계5동,2018-04-01,76.5,2,43.0,2,2,pm25
8,공릉2동,2018-04-01,116.6,4,68.9,2,2,pm10
9,상계2동,2018-04-01,87.4,4,59.7,2,2,pm10


In [10]:
# 모든 동 365일 각각 평균 미세먼지
dust = gu_total.groupby(['행정동','date'])['pm10', 'pm25'].mean().round(1)
dust

pm10  pm25
행정동 date                   
가회동 2018-04-01   98.3  56.5
    2018-04-02   87.8  36.1
    2018-04-03   93.4  36.1
    2018-04-04   19.4  10.3
    2018-04-05   17.1  11.6
...               ...   ...
혜화동 2019-03-27  118.4  71.9
    2019-03-28  112.9  67.6
    2019-03-29   55.8  26.4
    2019-03-30   62.0  39.0
    2019-03-31   67.8  47.5

[10220 rows x 2 columns]

In [11]:
# pm10_class
dust.loc[dust['pm10']< 31.0 , 'pm10_class'] = 0                           # 좋음(0): 0 < pm10 < 30
dust.loc[(dust['pm10']< 81.0) & (dust['pm10'] >= 31.0), 'pm10_class'] = 1    # 보통(1) :31 < pm10 < 80
dust.loc[(dust['pm10']< 151.0) & (dust['pm10']>= 81.0) , 'pm10_class'] = 2   # 나쁨(2): 81 < pm10 < 150
dust.loc[dust['pm10']>= 151.0 , 'pm10_class'] = 3                         # 매우나쁨(3): pm10 >= 151

# pm25_class
dust.loc[dust['pm25']< 16.0 , 'pm25_class'] = 0
dust.loc[(dust['pm25']< 36.0) & (dust['pm25'] >= 16.0), 'pm25_class'] = 1
dust.loc[(dust['pm25']< 76.0) & (dust['pm25'] >= 36.0) , 'pm25_class'] = 2
dust.loc[dust['pm25']>= 76.0 , 'pm25_class'] = 3

# pm_class(pm10과 pm25중 더 안좋은 것)
dust.loc[dust['pm10_class']< dust['pm25_class'] , 'pm_class'] = dust['pm25_class']
dust.loc[dust['pm10_class']>= dust['pm25_class'] , 'pm_class'] = dust['pm10_class']

# pm_class_info
dust.loc[dust['pm10_class']< dust['pm25_class'] , 'pm_class_info'] ='pm25'
dust.loc[dust['pm10_class']>= dust['pm25_class'] , 'pm_class_info'] ='pm10'

#
dust['pm10_class'] = dust['pm10_class'].astype('int')
dust['pm25_class'] = dust['pm25_class'].astype('int')
dust['pm_class'] = dust['pm_class'].astype('int')

dust = dust.reset_index()
dust = dust.rename(columns={'행정동':'DONG_CD', 'date':'STD_DD',
                            'pm_class': 'pm_class_today'})
dust

,DONG_CD,STD_DD,pm10,pm25,pm10_class,pm25_class,pm_class_today,pm_class_info
0,가회동,2018-04-01,98.3,56.5,2,2,2,pm10
1,가회동,2018-04-02,87.8,36.1,2,2,2,pm10
2,가회동,2018-04-03,93.4,36.1,2,2,2,pm10
3,가회동,2018-04-04,19.4,10.3,0,0,0,pm10
4,가회동,2018-04-05,17.1,11.6,0,0,0,pm10
5,가회동,2018-04-06,214.1,54.5,3,2,3,pm10
6,가회동,2018-04-07,53.7,29.4,1,1,1,pm10
7,가회동,2018-04-08,50.0,24.7,1,1,1,pm10
8,가회동,2018-04-09,64.3,45.7,1,2,2,pm25
9,가회동,2018-04-10,110.7,40.9,2,2,2,pm10
